In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data = np.loadtxt('./DataSet/data.csv', delimiter=',', unpack=True, dtype='float32')

In [7]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

Optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
train = Optimizer.minimize(cost, global_step=global_step)

sess = tf.Session()

In [15]:
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./DataSet/model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restores(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())    

In [16]:
for step in range(2):
    sess.run(train, feed_dict={X: x_data, Y: y_data})
    
    print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
saver.save(sess, './DataSet/model/dnn.ckpt', global_step=global_step)

0 0.79053766
1 0.72103333


'./DataSet/model/dnn.ckpt-2'

In [17]:
prediction = tf.argmax(model, 1)
actual = tf.argmax(Y, 1)

print(sess.run(prediction, feed_dict={X: x_data}))
print(sess.run(actual, feed_dict={Y: y_data}))

[0 1 2 0 0 2]
[0 1 2 0 0 2]


In [18]:
is_correct = tf.equal(prediction, actual)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print(sess.run(accuracy, feed_dict={X: x_data, Y: y_data}))

1.0


In [19]:
sess.close()